--------
### DeepDream

------

Deepdream is an artistic image modification technique that uses the representations learned by Convolutional Neural Networks. 


- The deepdream algorithm is very similar to Convnet's filter visualization algorithm discussed in Chapter 5.4. 
- ***Recap*** - For visualizing the convnets layers, we defined the loss function as the mean of all filter values. Now the goal was to maximize this. So we used the *keras.backend.Gradient* function to find the gradient between input and the loss we just defined. Since, we want to maximize this loss, we wrote the equation for Gradient ascent. We edited the input image repeatedly to maximize the given loss. The input image now represents what the filter is looking for or what activates the filter.

- With deep dream we will be maximizing the activation of entire layers rather than for a specific filter, thus mixing together visualizations of large number of features at once. 
- We will start with a existing image rather than a blank noisy image, thus the resulting effects latch on to preexisting visual patterns, distorting elements of the image in a somewhat srtistic fashion. 
- The input images are processed at different scales (called octaves), which improves the quality of visualization. 

---
***Deep Dream in Keras***

----

Let's start with the InceptionV3 model available as pretrained model in Keras.

In [22]:
from keras.applications import inception_v3
from keras import backend as K

In [23]:
### This command disables all training specific commands
K.set_learning_phase(0)

In [24]:
model = inception_v3.InceptionV3(weights = 'imagenet', include_top = False)

In [25]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_95 (Conv2D)               (None, None, None, 32 864                                          
____________________________________________________________________________________________________
batch_normalization_95 (BatchNor (None, None, None, 32 96                                           
____________________________________________________________________________________________________
activation_95 (Activation)       (None, None, None, 32 0                                            
___________________________________________________________________________________________

-----

Now our job is to define the loss function on which we will optimize the modify the input. This as discussed earlier will be defined as the layers mean value, and then we will apply gradient ascent with respect to the input image. 

Specifically we will maximize a weighted sum of the L2 norm of the set of high level layers. 

The set of layers we choose and the amount of contribution will have a huge impact on the final result. As they will decide the loss function we are trying to maximize. 

-----

***Setting up DeepDream Configuration***

-------

In [26]:
### Dictionary mapping layer names to quantifying how much a layer's output contributes to the loss function

layer_contributions = {
    'mixed2' : 0.2,
    'mixed3' : 3.,
    'mixed4' : 2.,
    'mixed5' : 1.5,
}

In [27]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [28]:
loss = K.variable(0.)

In [29]:
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    
    ### Used to get the multiplications of all dimensions for scaling
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    ### Now we calculate the L2 Norm by ignoring the corner pixels for better deepdream
    loss += coeff*K.sum(K.square(activation[:,2:-2, 2:-2,:]))/scaling

----

***Now we will setup the gradient ascent process***

------

In [30]:
dream = model.input

### Gradient of dream with respect to loss
grads = K.gradients(loss, dream)[0]

### Normalizing gradient
grads/= K.maximum(K.mean(K.abs(grads)), 1e-7)

### Setting up the function to retrieve values for loss and gradients, given an input
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

In [31]:
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_value = outs[1]
    return loss_value, grad_value

In [32]:
def gradient_ascent(x, iterations, step, max_loss = None):
    for i in range(iterations):
        loss_value, grad_value = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step*grad_value
    return x

------

***Let's define some auxillary functions before moving onto actual algorithm***

-------

In [33]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
              float(size[0])/img.shape[1],
              float(size[1])/img.shape[2],
              1)
    return scipy.ndimage.zoom(img, factors, order = 1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = inception_v3.preprocess_input(img)
    return img
    
### Undoes all the processing done by inception_v3 preprocessing
def deprocess_image(x):
    if K.image_data_format() == 'channel_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1,2,0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x,0,255).astype('uint8')
    return x

-----------
***Now we move onto the actual Deep Dream Algorithm***

-------

- First we define a list of scales (also called octaves) at which to process the images. 
- Each scale is larger than previous one by 40% i.e. 1.4 times. 
- We start by processing a small image and keep scaling it up. 
- For each successive scale we run Gradient Ascent and then scale the image up. 
- After scaling, we will reinject details from the original image back as we don't want to loose a lot of image detail. 

------

In [34]:
import numpy as np 

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20 

max_loss = 10.

base_image_path = 'image_scene.jpg'

In [35]:
img = preprocess_image(base_image_path)

In [36]:
### Defining the tuples of shapes which will be used in DeepDream Algorithm further

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1,num_octave):
    shape = tuple([int(dim/(octave_scale**i)) 
                  for dim in original_shape])
    successive_shapes.append(shape)

### Reversing the list 
successive_shapes = successive_shapes[::-1]
### Now in increasing order

In [37]:
### Copy of original image created and the image scaled down to smallest octave. 

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

In [38]:
### DeepDream loop, where we loop over all sizes and perform gradient ascent

for shape in successive_shapes:
    print('Processing Image Shape:', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                         iterations = iterations,
                         step = step,
                         max_loss = max_loss)
    ### Scales up the smaller version of the original image: Would be pixelated
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    ### Computes the high res of this size using Original image
    same_size_original = resize_img(original_img, shape)
    
    ### This is the information lost just because of scaling up 
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname = 'dream_scale_' + str(shape) + '.png')
save_img(img, fname = 'final_dream.png')

Processing Image Shape: (233, 319)
...Loss value at 0 : 1.7269647
...Loss value at 1 : 2.2122607
...Loss value at 2 : 2.8976164
...Loss value at 3 : 3.523584
...Loss value at 4 : 4.231632
...Loss value at 5 : 4.882864
...Loss value at 6 : 5.5317926
...Loss value at 7 : 6.10425
...Loss value at 8 : 6.669141
...Loss value at 9 : 7.2465844
...Loss value at 10 : 7.7482862
...Loss value at 11 : 8.272656
...Loss value at 12 : 8.800417
...Loss value at 13 : 9.3086405
...Loss value at 14 : 9.734537


/home/yash/anaconda3/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


Processing Image Shape: (326, 447)
...Loss value at 0 : 2.6883512
...Loss value at 1 : 3.9084873
...Loss value at 2 : 5.013199
...Loss value at 3 : 5.962499
...Loss value at 4 : 6.864773
...Loss value at 5 : 7.679968
...Loss value at 6 : 8.46775
...Loss value at 7 : 9.234631
...Loss value at 8 : 9.939697
Processing Image Shape: (457, 626)
...Loss value at 0 : 2.8869975
...Loss value at 1 : 4.1104484
...Loss value at 2 : 5.193458
...Loss value at 3 : 6.2044344
...Loss value at 4 : 7.1845
...Loss value at 5 : 8.173785
...Loss value at 6 : 9.187837
